## Load Packages

In [1]:
# Link to Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [56]:
# Connect to Earth Engine
import ee
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=och6gyUye6sXH7PcawfRvxVZtwv8gppLjKhKuKbn1Ms&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWjWyONylidZ0-szwVufkOFVRZtWjfufITEogD_F3n3GXHxSYrRXgDM

Successfully saved authorization token.


In [55]:
#!pip install geopandas
!pip install geojson

import os
from glob import glob
import numpy as np
import pandas as pd
import geopandas as gpd
import geojson

## Tidy up Datasets before Mapping/Plotting

### Group zonalStats dataframes by country

In [29]:
id_KHM = [541, 670]
id_VTM = [530,531,532,533,534,535,536,537,538,540,541]
id_THA = [655,656,657,665,666,667,668,669,670,671,672,695]
id_MYS = [651,658,659,660,662,663,664,618,619,626,627,628,629,630,631]
id_MMR = [655,693,694,695,696,697,698,699,700,701,702]

In [50]:
def filterRegion(list_id, country):
  """ Merge GeoJSONs and Subset to one country

  Parameters
  ----------
  list_id: list
    List of parcel ids
  country: string
    Country Name
  
  Returns
  ----------
  Exported GeoJSON of one country
  """
  os.chdir("/content/drive/MyDrive/THESIS_AQUAPONDS/gdf_zonstats/0.Initial/")
  files = [glob(os.path.join(os.getcwd(), '*'+str(i)+'*')) for i in list_id]
  files_flat = [item for sublist in files for item in sublist]
  gdfs = [gpd.read_file(i) for i in files_flat]
  gdfs_filtered = [i[i['NAME_0']==country] for i in gdfs]
  gdf = pd.concat([i for i in gdfs_filtered])

  # Data type of zonStats columns to "uint8"
  keys = [i for i in gdf.columns if 'majority' in i]
  values = [np.uint8]*len(keys)
  dictCol = dict(zip(keys, values))
  gdf = gdf.astype(dictCol)

  outPath = '/content/drive/MyDrive/THESIS_AQUAPONDS/gdf_zonstats/1.by_country/'
  return gdf.to_file(os.path.join(outPath, "zonStats_"+str(country)+".geojson"), driver = 'GeoJSON')

In [51]:
# Write file
#filterRegion(id_KHM, "Cambodia")
filterRegion(id_VTM, "Vietnam")
filterRegion(id_THA, "Thailand")
filterRegion(id_MYS, "Malaysia")
filterRegion(id_MMR, "Myanmar")

In [54]:
# Draft
#os.chdir("/content/drive/MyDrive/THESIS_AQUAPONDS/gdf_zonstats/0.Initial/")
#files = [glob(os.path.join(os.getcwd(), '*'+str(i)+'*')) for i in id_cambodia]
#files_flat = [item for sublist in files for item in sublist]
#gdfs = [gpd.read_file(i) for i in files_flat]
#gdfs_filtered = [i[i['NAME_0']=="Cambodia"] for i in gdfs]
#gdf_KHM = pd.concat([i for i in gdfs_filtered])

#keys = [i for i in gdf_KHM.columns if 'majority' in i]
#values = [np.uint8]*len(keys)
#dictCol = dict(zip(keys, values))
#gdf_KHM = gdf_KHM.astype(dictCol)

### Calculate Pond Area, adding as Attributes (in GEE)

In [58]:
os.chdir("/content/drive/MyDrive/THESIS_AQUAPONDS/gdf_zonstats/1.by_country/")
with open('zonStats_Cambodia.geojson') as f:
  KHM_geojson = geojson.load(f)

In [66]:
# Load GeoJSON as feature collection
fc_KHM = ee.FeatureCollection(KHM_geojson[0:10])

In [75]:
# Pond Area in square meters
fc_KHM_area = fc_KHM.map(lambda feature: feature.set({'Area': feature.area()}))

### Transfer Geometry: Polygon --> Point (in GEE?)

In [76]:
# Get Centroid of Pond Polygons
fc_KHM_cent = fc_KHM_area.map(lambda feature: feature.centroid())

In [80]:
# Export Feature Collection as GeoJSON
task_config = {
        'description': 'zonStats_KHM_tidy',
        'fileNamePrefix': 'zonStats_KHM_tidy',
        'folder': '2.area_centroid',
        'fileFormat': 'GeoJSON'}

task = ee.batch.Export.table.toDrive(fc_KHM_cent, **task_config)
task.start()

In [85]:
task.status()

{'attempt': 1,
 'creation_timestamp_ms': 1647879428040,
 'description': 'zonStats_KHM_tidy',
 'destination_uris': ['https://drive.google.com/#folders/1x9_4kzq3hUPmQslznFfoutlpa1sqw9_7'],
 'id': 'AHYCZSZQY2NN5Z7SNO6QJUIQ',
 'name': 'projects/earthengine-legacy/operations/AHYCZSZQY2NN5Z7SNO6QJUIQ',
 'start_timestamp_ms': 1647879508891,
 'state': 'COMPLETED',
 'task_type': 'EXPORT_FEATURES',
 'update_timestamp_ms': 1647879510949}